# Visualize White Island Rainfall Data For Crater Lake Level Assessment

Rainfal data stored in a series of (monthly) CSV files scraped from Metconnect (using a separate notebook).
Lake level data from FITS.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

**rainfall data**

In [ ]:
files = sorted(glob.glob('*.csv'))

df = pd.DataFrame()
for f in files:
    month = pd.read_csv(f, index_col='Date', parse_dates=True)
    df = df.append(month)

**lake level data**

In [ ]:
url = 'https://fits.geonet.org.nz/observation?siteID=WI200&typeID=z&methodID=img&days=365'
ll = pd.read_csv(url, parse_dates=True, index_col='date-time')

In [ ]:
fig,(ax1) = plt.subplots(nrows=1,ncols=1, figsize=(15,7.5))
ax1.bar(df.index, df['Rainfall'])
ax1.set_ylabel('rainfall (mm)')
plt.savefig('rainfall.png')

# Lake Catchment Analysis

Rainfall and Water Level Rise
--

During 2018, the lake has risen about 10 m.

During the same period, there has been about 1 m of direct rainfall on the lake. Allowing for ~2.5 month of missing data, I assume 1.3 m of rainfall on the lake.

Catchment Area
--

This is about 5.5 times the area of the lake. The GE image is for 2011, but the lake area won't have increased by more than 10% in the 2016 eruption???
![](lake_catchment_area.png)

# Volume - Depth

Cam's table

In [ ]:
vd = pd.read_excel('WI_Volume_with_depth.xlsx')
vd.head()

**Fit oder 2 polynomial to height vs volume**

In [ ]:
import numpy as np

X = vd['MASL'].values
y = vd['Volume'].values

coefs = np.polyfit(X, y, 2)
p = np.poly1d(coefs)

fig,(ax1) = plt.subplots(nrows=1,ncols=1, figsize=(7,7))
ax1.plot(vd['MASL'], vd['Volume'], label='obs')
ax1.set_ylabel('Volume (m^3)')
ax1.set_xlabel('Height above sea level (m)')
ax1.plot(vd['MASL'], p(X), color='red', label='fit')
plt.legend()

**Calculate volume for each observed height**

In [ ]:
ll['volume'] = p(ll[' z (m)'])

**Plot volume**

In [ ]:
fig,(ax1) = plt.subplots(nrows=1,ncols=1, figsize=(15,5))
ax1.plot(df.index, np.cumsum(df['Rainfall']))
ax1.set_ylabel('cumulative rainfall (mm)')

ax2 = ax1.twinx()
ax2.plot(ll.index, (ll['volume']), marker='o', color='#ff7f0e')

ax2.set_ylabel('lake volume (m$^3$)', color='#ff7f0e')

plt.savefig('volume-rainfall.png')

**Plot level**

In [ ]:
fig,(ax1) = plt.subplots(nrows=1,ncols=1, figsize=(15,5))
ax1.plot(df.index, np.cumsum(df['Rainfall']))
ax1.set_ylabel('cumulative rainfall (mm)')

ax2 = ax1.twinx()
ax2.plot(ll.index, (ll[' z (m)']), marker='o', color='#ff7f0e')

ax2.set_ylabel('lake level (m)', color='#ff7f0e')

plt.savefig('level-rainfall.png')